In [10]:
from bs4 import BeautifulSoup
import lxml
import re
import pandas as pd

In [11]:
r = re.compile(r'R(\d+)C(\d+)')

def make_soup(path):
    with open(path, encoding='utf-8') as f:
        raw_html = f.read()
        f.close()
        soup = BeautifulSoup(raw_html, 'lxml')
        return soup

def save_pkl(df, name='df.pkl'):
    df.to_pickle(name)

def create_df(rows, columns, index='№ п/п'):
    adm_df = pd.DataFrame(rows, columns=columns).set_index(index)
    return adm_df

def parse_data(soup, cid: str) -> list:
    results = []

    found = soup.find(cid=cid).previous_sibling
    for sib in list(found.next_siblings):
        if sib.string == '\n':
            continue
        location = sib.attrs.get('cid')
        contents = sib.string

        if contents is None:
            contents = [desc.string for desc in sib.descendants]
            contents = [item.string.rstrip() for item in contents if item is not None]
            # This block avoids cases for "contents = []", makes pandas pretty
            if len(contents) == 0:
                contents = ''
        results.append((str(location), str(contents)))

    return results

def make_rows(extracted_data: list, num_cols: int) -> list:
    results = []
    for row in range(0, len(extracted_data), num_cols):
        each_row_list = extracted_data[row:row + num_cols]
        each_row_data = [data[1] for data in each_row_list]
        # Avoids a case where the last row is (usually) empty
        if each_row_data and len(each_row_data[0]) > 0:
            results.append(each_row_data)

    return results

def make_columns(soup):
    ''' Analogue search for columns '''
    max_column = max(int(r.search(div['cid']).group(2)) for div in soup.find_all(cid=r))
    temp_labels = []
    for i in range(1, max_column + 1):
        if not soup.find(cid='R{}C{}'.format(7, i)) is None:
            temp_labels.append(soup.find(cid='R{}C{}'.format(7, i)).div.text)
    for i in temp_labels:
      if i not in temp_labels:
        temp_labels.append(i)
            
    print(temp_labels)
    return temp_labels

In [12]:
FILE_PATH = 'Программа двух дипломов НИУ ВШЭ и Университета Кёнхи Экономика и политика в Азии.html'

In [13]:
soup = make_soup(FILE_PATH)

In [14]:
columns_plus = parse_data(soup, cid="R7C1")
data = make_rows(columns_plus, num_cols=44)
cols = data[0]
del data[0]
rows = data

In [15]:
df = create_df(rows, cols)
df

,Регистрационный номер,"Фамилия, имя, отчество",Подлинник/Копия документа \nоб образовании,Медаль / диплом с отличием,Право поступления без вступительных испытаний,"Поступление на места в рамках квоты \nдля лиц, имеющих особое право",Поступление на места в рамках квоты\nцелевого приема,Наличие согласия на зачисление,Дата и время предоставления согласия,История,...,Все выбранные конкурсы,Олимпиады,Приказ о зачислении,Основание зачисления / выбытия,Последний результат опроса,Статус соотечественника,Гражданство,Гражданство СНГ,Имеется легализация / Имеется \nпризнание,Второй иностранный язык
№ п/п,,,,,,,,,,,,,,,,,,,,,
1,6437,Абрамова Софья Владимировна,Копия,+,,-,-,Нет,,98,...,"МО_Л, ЭКПА, Ант (Ист), ИСТ",,,,,Российская Федерация,,Да,Нет/Нет,
2,1279,Агаркова Мария Александровна,Копия,+,,-,-,Нет,,54,...,"Вост, ЭКПА",,,,,Российская Федерация,,Да,Нет/Нет,
3,13467,Азаров Георгий Сергеевич,Копия,-,,-,-,Нет,,58,...,"Вост, ЭКПА",,,,,Российская Федерация,,Да,Нет/Нет,
4,7326,Алексеев Александр Дмитриевич,Подлинник,+,,-,-,Да,03.08.2020 0:00,94,...,"Вост, ЭКПА (подл.)",,,,,Российская Федерация,,Да,Нет/Нет,
5,4596,Андрусенко Софья Владимировна,Копия,-,,-,-,Нет,,,...,"ПРАД, ЭКПА, МИРЭК",,,,,Российская Федерация,,Да,Нет/Нет,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,7765,Шуленина Александра Васильевна,Подлинник,-,,-,-,Да,03.08.2020 0:00,68,...,"Вост, ЭКПА (подл.), МО_Л",,,,,Российская Федерация,,Да,Нет/Нет,
184,2464,Шурховецкая Людмила Рубеновна,Копия,-,,-,-,Нет,,83,...,"Вост, МО_Л, ЭКПА",,,,,Российская Федерация,,Да,Нет/Нет,
185,18372,Эрендженова Даяна Батыровна,Подлинник,+,,-,-,Да,12.08.2020 11:37,92,...,"ЭКПА (подл.), Вост",,,,,Российская Федерация,,Да,Нет/Нет,
